## Coleta de dados da Web

Demonstração de alguns recursos importantes para essa tarefa

_Victor Emanuel Perticarrari Osório_

# Carregando Bibliotecas

In [1]:
#Instalação das apis - caso não tenha instalado
!pip install TwitterAPI
!pip install simplejson
!pip install pandas
!pip install pymongo
!pip install beautifulsoup4
!pip install selenium

# Execício 1

Faça uma adaptação no coletor de tweets para coletar qualquer tipo de tweet compartilhado no
Brasil. Dica: veja a documentação da biblioteca TwitterApi. Faça uma coleta por alguns minutos,
salvando o resultado em um arquivo ou em um banco de dados.

In [4]:
from TwitterAPI import TwitterAPI


## Substitua os valores abaixo
## Crie chaves em https://developer.twitter.com/en/portal/apps
twitter_api = TwitterAPI(consumer_key='XXXX',
                         consumer_secret='XXXX',
                         access_token_key='XXXX',
                         access_token_secret='XXXX')
print(twitter_api)

Os dados serão armazenados no MongoDB como foram recebidos.

Instale um MongoDB sem restrição de acesso ou execute o container abaixo:

```bash
docker run --name tweets -p 27017:27017 -d mongo:4
```

In [5]:
import time
## Tutorial pandas: https://pandas.pydata.org/docs/getting_started/intro_tutorials/01_table_oriented.html#min-tut-01-tableoriented
import pandas as pd

from pymongo import MongoClient
import pymongo

## Coordenadas encontradas em https://gist.github.com/graydon/11198540
filters = {"locations": "-73.9872354804, -33.7683777809, -34.7299934555, 5.2444863956"}

tweet_stream = twitter_api.request('statuses/filter', filters).get_iterator()
TEMPO_TOTAL_EM_MIN = 0.5
CONNECTION_STRING = "mongodb://localhost:27017/tweets"
client = MongoClient(CONNECTION_STRING)
database = client['dados']
tweets_collections = database["tweets"]
df = None
t_inicial = time.time() ## Inicio em segundos
for tweet in tweet_stream:
    try:
        ## Usando id do tweet para evitar tweets repetidos
        tweet['_id'] = tweet['id']
        del tweet['id']
        tweets_collections.insert_one(tweet)
    except Exception as ex:
        print(ex)
    t_total = time.time() - t_inicial
    if t_total > TEMPO_TOTAL_EM_MIN * 60:
        break
print("Tweets coletados: {}".format(tweets_collections.count_documents({})))

Tweets coletados: 4316


# Exercício 2
Com base no dataset gerado na questão 1, faça:
  1. Separe todas as hashtags usadas nos Tweets.
  2. Descubra a porcentagem de tweets com hashtags.
  3. Identifique a hashtag mais popular.

In [6]:
AGG_HASHTAGS_HISTOGRAM = [{'$project': {'entities.hashtags.text': 1}}, {'$match': {'entities.hashtags': {'$not': {'$size': 0}}}}, {'$project': {'hashtag': '$entities.hashtags.text'}}, {'$unwind': '$hashtag'}, {'$group': {'_id': '$hashtag', 'frequence': {'$sum': 1}}}, {'$sort': {'frequence': -1}}]
AGG_HASHTAGS_TODAS = [{'$project': {'entities.hashtags.text': 1}}, {'$match': {'entities.hashtags': {'$not': {'$size': 0}}}}, {'$project': {'hashtag': '$entities.hashtags.text'}}, {'$unwind': '$hashtag'}, {'$group': {'_id': None, 'hashtags': {'$addToSet': '$hashtag'}}}, {'$unwind': '$hashtags'}, {'$sort': {'hashtags': 1}}, {'$group': {'_id': None, 'hashtags': {'$push': '$hashtags'}}}]
AGG_HASHTAG_INFO = [{'$project': {'_id': '$_id', 'hashtag': {'$cond': [{'$gt': [{'$size': '$entities.hashtags'}, 0]}, 1, 0]}}}, {'$group': {'_id': None, 'total': {'$sum': 1}, 'total_hash': {'$sum': '$hashtag'}}}]

## 1. Separe todas as hashtags usadas nos Tweets
todas_hashtags = tweets_collections.aggregate(AGG_HASHTAGS_TODAS).next()
print("1. Todas hashtags: {}".format(todas_hashtags['hashtags']))

info_hashtags = tweets_collections.aggregate(AGG_HASHTAG_INFO).next()
print("2. Tweets com hashtag: {:0.2f}%".format((info_hashtags['total_hash'] / info_hashtags['total']) * 100))

popular_hashtag = tweets_collections.aggregate(AGG_HASHTAGS_HISTOGRAM).next()
print("3. Mais popular: tag={} ocorrência={}".format(popular_hashtag["_id"], popular_hashtag["frequence"]))

1. Todas hashtags: ['1MDoSport', '2022candidatocomecandocom13naoconfirmar', '2022gameoverlulismo', 'AM', 'AMExSPFC', 'ATENCIÓN', 'AYUDEMOS', 'AlemDaIlusao', 'ApruebaSeraHermoso', 'Apruebazo', 'Apruebo', 'AprueboFeliz', 'AprueboSinMiedo', 'AprueboUnChileMejor', 'Arequipa', 'B22', 'BUNDAAWARDS', 'BUNDASAWARDS', 'Bogotá', 'Bolsonaro22', 'BolsonaroNoPrimeiroTurno', 'BolsonaroReeleito', 'BritneyIsComing', 'BuenasNoches', 'BuenasNochesATodos', 'CHILE', 'CHVNoticias', 'COLABOREMOS', 'COMPARTAMOS', 'CantaConmigoAhora', 'Chano', 'ChvNoticias', 'CiroComAndreMarinho', 'CiroComAndréMarinho', 'CiroPresidente12', 'ConhecerParaPreservar', 'ConvivenciaEscolar', 'CopaChileEasy', 'CopaDoBrasil', 'CopadoMundo', 'CoquimboUnido', 'Cordoba', 'DailyPikminBloom', 'DaleRaya', 'Dentist', 'DesisteCiro', 'DisciplinaTour', 'Disney', 'EDLP', 'EEUU', 'EdegarPrettoGovernador13', 'EuNunca3', 'ForaBolsonaro', 'FueraAzulAzul', 'GabrielMonteiroCassado', 'GabrielMonteiroNaCadeia', 'Gabrielmonteiro', 'HOLYFVCK', 'HoldMeClo

# Exercício 3

Faça um coletor que capture as reviews da página principal de:
https://www.apontador.com.br/local/sp/campinas/faculdades_e_universidades/88Q2E372/
unicamp.html.

In [10]:
import requests
from bs4 import BeautifulSoup

req = requests.get("https://www.apontador.com.br/local/sp/campinas/faculdades_e_universidades/88Q2E372/unicamp.html")
if req.status_code >= 200 and req.status_code < 300:
    print("Sucesso")
    soup = BeautifulSoup(req.content, 'html.parser')
    comentarios = soup.find_all(name='div',attrs={'class':"store-comments__item"})
    for comentario in comentarios:
        print("""
Nome:        {}
Commentário: {}""".format(comentario.find(attrs={'class':"name"}).get_text(),
                          comentario.find(attrs={'class':"store-comments__text"}).get_text()))
else:
    print("ERRO! {}".format(req.status_code))

Sucesso

Nome:        Renan Cunha De Sousa
Commentário:  Unicamp é umas das mais cobiçadas universidades de Campinas e região, devido a sua imensa qualidade, por ser federal se torna a melhor faculdade da região. Sem contar o seu Campus que é sensacional, como se fosse uma cidade, possui quadras com prédios, até uma linha de ônibus circular que permanece dentro do campus apenas devido o seu tamanho. A universidade conta também com o Hospital das Clínicas, gratuito pelo SUS e administrar o hospital de Sumaré. As provas para ingressar na faculdade são por fases com respostas alternativas e requisitando cerca de 3 redações o número de candidatos por vaga é bem alto principalmente no curso de medicina, o problema fica no pagamento do vestibular que é cerca de 120~140 reais para poder executa-la. A graduação é em tempo integral com isso a faculdade possui centro de pesquisas, onde é possível fazer iniciações cientificas ou mini empresas, para que os alunos tenham uma breve praticidade no me

# Exercício 4

Crie uma aplicação que usa o Selenium para abrir o site https://www.uol.com.br/ e rolar a página
para baixo até o fim do conteúdo.

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome(executable_path='./chromedriver')


driver.get("https://www.uol.com.br/")  
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

C:\Users\victoros\AppData\Local\Temp\ipykernel_14040\2169359952.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver')
